In [ ]:
import os
import glob
import json
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../har_research")

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import clip
from PIL import Image
from IPython.display import display

In [ ]:
from har_research.image_scan import iter_image_entries

In [ ]:
model, preprocess = clip.load("ViT-B/32")

In [ ]:
labels = ["Charlie", "Charlie Manson", "Charles Manson"]
text = clip.tokenize(labels).cuda()

image = Image.open("/home/bergi/Pictures/__diverse/CHARLIE2.GIF")
image = preprocess(image).unsqueeze(0).cuda()

with torch.no_grad():
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.cpu().numpy()[0]
    
print(probs)

In [ ]:
def store_har_image_features(glob_pattern: str, feature_filename: str):
    with open(feature_filename, "w") as fp:
        for e, image in iter_image_entries(glob_pattern):
            if image.size[0] < 10:
                continue
            try:
                image = preprocess(image).unsqueeze(0).cuda()
                with torch.no_grad():
                    features = model.encode_image(image)
            except Exception as ex:
                print(e["request"]["url"], ":", ex)
                continue

            line = '{"url":"%s","features":[%s]}\n' % (
                e["request"]["url"],
                ",".join(str(f)[:8] for f in features.cpu().numpy()[0])
            )
            #print(line)
            #break
            fp.write(line)

    #print(e["request"]["url"])
    
store_har_image_features(
    "../har_research/automatic/recordings/*/*.json",
    "clip-image-features.ndjson"
)